### Дополнительный инструментарий

In [1]:
# Песочница
#data0 = data.copy()
#data0.loc[[1745,4127]]

In [2]:
# Функция импорта из *.csv
def load_from_csv(source1, source2, path2='https://code.s3.yandex.net/datasets/', sep=','):
    """
        Функция организации загрузки данных в датафрейм из файла *.csv
        на входе:
            source1 - путь + файл *.csv на локальном компьютере
            source2 - файл *.csv
            path2 - путь (по умолчанию 'https://code.s3.yandex.net/datasets/')
            sep - разделитель (по умолчанию ',')
        на выходе:
            df (DataFrame)
    """
    try:
        df = pd.read_csv(source1, sep=sep)
    except ValueError:
        print('Fail - ValueError!')
    except Exception:
        print('Fail - загрузка из основного источника')
        print('Попытка загрузки из резервного источника')
        df = pd.read_csv(str(path2+source2), sep=sep)
        print(f'OK - загрузка из резервного источника: "{str(path2+source2)}"')
    else:
        print(f'OK - загрузка из основного источника: "{source1}"')
    finally:
        print('Конец.') 
    return df

#df = df.drop([''], axis=1)
#df.drop(index=index_list, inplace=True)
#df.to_csv('*.csv', index=False)

In [3]:
# Функция дополнительного функционала для pd.hist() 
def grafix(df_hist, x_label='', y_label='', title=''):
    for ax in df_hist.flatten():
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        ax.set_title(title)
    
    plt.show()

In [4]:
# Функция вывода диаграмм рассеяния и плотности
def plot_scatter_matrix(df, plotSize, textSize):
    # отбросить пропуски
    df = df.dropna()
    # оставить числовые столбцы
    df = df.select_dtypes(include=[np.number])
    # оставить столбцы с заданным количеством уникальных значений
    df = df[[col for col in df if df[col].nunique() > 1]]
    columnNames = list(df)
    # ограничение на количество столбцов в матрице
    if len(columnNames) > 14:
        columnNames = columnNames[:14]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2),
                          xycoords='axes fraction', ha='right', va='center', size=textSize)
    plt.suptitle('Диаграммы рассеяния и плотностей', fontsize=32)
    plt.show()

In [5]:
def first_view(df): 
    """
    функция для отображения общей информации
    
    """
    display(df.head(5));
    df.info();
    display(df.describe().style.format('{:.1f}'));
    display(df.corr(method='spearman').style.format('{:.2f}').background_gradient('coolwarm'));
    df.hist(figsize=(12, 15), bins=50);
    plot_scatter_matrix(df, 30, 15);

In [6]:
# функция вывода информации о столбце
def column_value_test(ds, test='un'):
    """
        Функция вывода информации о столбце
        'un' - unique(); 'na' - isna(); 'du' - duplicated()
    """
    if test == 'un':
        return print(f'Кол-во уникальных значений в столбце: "{ds.name}": {len(ds.unique())}/{len(ds)}')
    elif test == 'na':
        return print(f'Кол-во пропущенных значений в столбце: "{ds.name}": {ds.isna().sum()}/{len(ds)}')
    elif test == 'du':
        return print(f'Кол-во дубликатов в столбце:\n{ds.name}: {ds.duplicated().sum()}/{len(ds)}')
    else:
        return print(f'Error!')
    
#column_value_test(data.iloc[:, 0])
#column_value_test(data.iloc[:, 0], test='na')
#column_value_test(data.iloc[:, 0], test='du')

In [7]:
# Функция разделеления данных на выборки: тренировочную, валидационную, тестовую
def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.6, frac_valid=0.2, frac_test=0.2,
                                         random_state=12345, prnt=True):
    '''
    Splits a Pandas dataframe into three subsets (train, val, and test)
    following fractional ratios provided by the user, where each subset is
    stratified by the values in a specific column (that is, each subset has
    the same relative frequency of the values in the column). It performs this
    splitting by running train_test_split() twice.

    Parameters
    ----------
    df_input : Pandas dataframe
        Input dataframe to be split.
    stratify_colname : str
        The name of the column that will be used for stratification. Usually
        this column would be for the label.
    frac_train : float
    frac_valid : float
    frac_test  : float
        The ratios with which the dataframe will be split into train, val, and
        test data. The values should be expressed as float fractions and should
        sum to 1.0.
    random_state : int, None, or RandomStateInstance
        Value to be passed to train_test_split().

    Returns
    -------
    X_train, X_val, X_test, y_train, y_valid, y_test:
        Dataframes containing the six splits.
    '''

    if frac_train + frac_valid + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_valid, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    # Contains dataframe without stratify column
    X = df_input.drop([stratify_colname], axis=1)
    
    # Dataframe of just the column on which to stratify
    y = df_input[[stratify_colname]]

    if frac_test > 0:
        # Split X and y into train and temp.
        X_train, X_temp, y_train, y_temp = train_test_split(X,
                                                            y,
                                                            stratify=y,
                                                            test_size=(1.0 - frac_train),
                                                            random_state=random_state)

        # Split the temp X and y into val and test.
        relative_frac_test = frac_test / (frac_valid + frac_test)
        X_valid, X_test, y_valid, y_test = train_test_split(X_temp,
                                                            y_temp,
                                                            stratify=y_temp,
                                                            test_size=relative_frac_test,
                                                            random_state=random_state)
    else:
        X_train, X_valid, y_train, y_valid = train_test_split(X,
                                                              y,
                                                              #stratify=y,
                                                              test_size=(1.0 - frac_train),
                                                              random_state=random_state)
        X_test = {}
        y_test = {}

    assert len(df_input) == len(X_train) + len(X_valid) + len(X_test)
    assert len(df_input) == len(y_train) + len(y_valid) + len(y_test)
    
    if prnt:
        print('dataframe:      ', df_input.shape)
        print('features_train: ', X_train.shape)
        print('features_valid: ', X_valid.shape)
        if frac_test > 0:
            print('features_test:  ', X_test.shape)
        print('target_train:   ', y_train.shape)
        print('target_valid:   ', y_valid.shape)
        if frac_test > 0:
            print('target_test:    ', y_test.shape)
        
    if frac_test > 0:
        return X_train.squeeze(), X_valid.squeeze(), X_test.squeeze(),\
               y_train.squeeze(), y_valid.squeeze(), y_test.squeeze()
    elif frac_test == 0:
        return X_train.squeeze(), X_valid.squeeze(), y_train.squeeze(), y_valid.squeeze()
    else:
        return "Error!"

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />    
Жестокий кодинг )

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Жизнь с учёбой ожесточили.</font>
</div>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Данные геологоразведки трёх регионов находятся в файлах: 

    https://code.s3.yandex.net/datasets/geo_data_0.csv
    https://code.s3.yandex.net/datasets/geo_data_1.csv
    https://code.s3.yandex.net/datasets/geo_data_2.csv
    
Условия задачи:

    Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
    При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
    Бюджет на разработку скважин в регионе — 10 млрд рублей.
    При нынешних ценах один баррель сырья приносит 450 рублей дохода.
    Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
    После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%.
    Среди них выбирают регион с наибольшей средней прибылью.

**Объект исследования** — три нефтяных региона.

**Цель исследования** — определить регион, где добыча принесёт наибольшую среднюю прибыль при вероятности убытков менее 2,5%.

**План исследования:** 

    I. Загрузка и обзор данных из файлов *.csv.
    II. Подготовка дополнительного инструментария.
    1. Разбиение данных на обучающую и валидационную выборки (соотношение 3:1).
    2. Обучение модели и предсказание на валидационной выборке.
    3. Средний запас разведанного, предсказанного сырья и RMSE модели.
    4. Подготовка к расчёту прибыли.
    5. Разработка функции для расчёта прибыли.
    6. Расчёт средней прибыли, 95%-й доверительного интервала и риска убытков.
    7. Отбор моделей с вероятностью убытков менее 2,5%. Модель с наибольшим значением средней прибыли.  
    III. Общий вывод.
      
**Описание данных:**

Признаки:

    id — уникальный идентификатор скважины
    f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы)

Целевой признак:

    product — объём запасов в скважине (тыс. баррелей)

### Загрузка и подготовка данных

In [8]:
# импорт библиотек
import pandas as pd
import numpy as np
from numpy.random import RandomState
import matplotlib.pyplot as plt
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

In [9]:
# настройки отображения данных
#matplotlib.rcParams.update({'font.size': 12})
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [10]:
# ГЛОБАЛЬНЫЕ ПЕРЕМЕННЫЕ
# включение всех функций
RELEASE_GLOBAL = False
# генератор случайных чисел с ожидаемым числом
STATE_GLOBAL = RandomState(12345)

In [11]:
# вывод предупреждений
import warnings
if RELEASE_GLOBAL:
    warnings.filterwarnings(action='ignore')

In [12]:
# Загрузка данных
df1 = load_from_csv(source1='./datasets/geo_data_0.csv', source2='geo_data_0.csv', sep=',')
df2 = load_from_csv(source1='./datasets/geo_data_1.csv', source2='geo_data_1.csv', sep=',')
df3 = load_from_csv(source1='./datasets/geo_data_2.csv', source2='geo_data_2.csv', sep=',')

OK - загрузка из основного источника: "./datasets/geo_data_0.csv"
Конец.
OK - загрузка из основного источника: "./datasets/geo_data_1.csv"
Конец.
OK - загрузка из основного источника: "./datasets/geo_data_2.csv"
Конец.


### Первый "взгляд"

#### Первый регион

In [13]:
if RELEASE_GLOBAL:
    first_view(df1)

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Я ревьюер простой: вижу гистограммы - пишу зелёный комментарий. А уж коэффициенты корреляции - тем более важная вещь в нашем проекте, ведь по ТЗ у нас линейная модель, нам важно не столкнуться с мультиколлинеарностью.

In [14]:
# Проверка в столбцах на уникальные значения, пропуски, дубликаты
for i in range(len(df1.columns)):
    column_value_test(df1.iloc[:, i], test='un')
    column_value_test(df1.iloc[:, i], test='na')
    column_value_test(df1.iloc[:, i], test='du')

Кол-во уникальных значений в столбце: "id": 99990/100000
Кол-во пропущенных значений в столбце: "id": 0/100000
Кол-во дубликатов в столбце:
id: 10/100000
Кол-во уникальных значений в столбце: "f0": 100000/100000
Кол-во пропущенных значений в столбце: "f0": 0/100000
Кол-во дубликатов в столбце:
f0: 0/100000
Кол-во уникальных значений в столбце: "f1": 100000/100000
Кол-во пропущенных значений в столбце: "f1": 0/100000
Кол-во дубликатов в столбце:
f1: 0/100000
Кол-во уникальных значений в столбце: "f2": 100000/100000
Кол-во пропущенных значений в столбце: "f2": 0/100000
Кол-во дубликатов в столбце:
f2: 0/100000
Кол-во уникальных значений в столбце: "product": 100000/100000
Кол-во пропущенных значений в столбце: "product": 0/100000
Кол-во дубликатов в столбце:
product: 0/100000


<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Молодец, что нашёл неявные дубликаты, это далеко не все делают

In [15]:
# дубли в столбце 'id'
df1[df1['id'].duplicated()].head(11)

,id,f0,f1,f2,product
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
66136,74z30,1.084962,-0.312358,6.990771,127.643327
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
75715,Tdehs,0.112079,0.430296,3.218993,60.964018
90815,fiKDv,0.049883,0.841313,6.394613,137.346586
92341,TtcGQ,0.110711,1.022689,0.911381,101.318008
97785,bsk9y,0.378429,0.005837,0.160827,160.637302


In [16]:
df1.value_counts('product', normalize=False, sort=False)

product
0.000000      1
0.004022      1
0.006114      1
0.009428      1
0.021781      1
             ..
185.352015    1
185.354980    1
185.355615    1
185.362690    1
185.364347    1
Length: 100000, dtype: int64

Наблюдения:

- признаки 'f0' и 'f1' имеют вид сложения нескольких нормальных распределений
- корреляция между 'f0' и 'f1' около 0,5; eсть корреляции с 'product'
- признак 'f2' имеет нормальное распределение
- признак 'product' имеет распределение смешанного вида (нормальное и равномерное) аналогичное с df3
- корреляция 'f2' c 'product' около 0,5
- дубли в столбце 'id', возможно, возникли в результае коллизий хеш-функции

#### Второй регион

In [17]:
if RELEASE_GLOBAL:
    first_view(df2)

In [18]:
# Проверка в столбцах на уникальные значения, пропуски, дубликаты
for i in range(len(df2.columns)):
    column_value_test(df2.iloc[:, i], test='un')
    column_value_test(df2.iloc[:, i], test='na')
    column_value_test(df2.iloc[:, i], test='du')

Кол-во уникальных значений в столбце: "id": 99996/100000
Кол-во пропущенных значений в столбце: "id": 0/100000
Кол-во дубликатов в столбце:
id: 4/100000
Кол-во уникальных значений в столбце: "f0": 100000/100000
Кол-во пропущенных значений в столбце: "f0": 0/100000
Кол-во дубликатов в столбце:
f0: 0/100000
Кол-во уникальных значений в столбце: "f1": 100000/100000
Кол-во пропущенных значений в столбце: "f1": 0/100000
Кол-во дубликатов в столбце:
f1: 0/100000
Кол-во уникальных значений в столбце: "f2": 100000/100000
Кол-во пропущенных значений в столбце: "f2": 0/100000
Кол-во дубликатов в столбце:
f2: 0/100000
Кол-во уникальных значений в столбце: "product": 12/100000
Кол-во пропущенных значений в столбце: "product": 0/100000
Кол-во дубликатов в столбце:
product: 99988/100000


In [19]:
# дубли в столбце 'id'
df2[df2['id'].duplicated()].head()

,id,f0,f1,f2,product
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [20]:
# дубликаты в строках с выборочными столбцами
df2[['id', 'product']].duplicated().sum()

0

In [21]:
# перечень категорий в признаке 'product'
df2.value_counts('product', normalize=False, sort=False) 

product
0.000000      8235
3.179103      8337
26.953261     8468
30.132364     8306
53.906522     8472
57.085625     8390
80.859783     8320
84.038886     8431
107.813044    8201
110.992147    8303
134.766305    8304
137.945408    8233
dtype: int64

In [22]:
# разделим на два явных подмножества
indexes = df2.value_counts('product', normalize=False, sort=False).index
indexes1 = indexes[[0,2,4,6,8,10]]  
indexes2 = indexes[[1,3,5,7,9,11]] 

In [23]:
df2_1 = df2.query('product in @indexes1')
df2_1.value_counts('product', normalize=False, sort=False)

product
0.000000      8235
26.953261     8468
53.906522     8472
80.859783     8320
107.813044    8201
134.766305    8304
dtype: int64

In [24]:
df2_2 = df2.query('product in @indexes2')
df2_2.value_counts('product', normalize=False, sort=False)

product
3.179103      8337
30.132364     8306
57.085625     8390
84.038886     8431
110.992147    8303
137.945408    8233
dtype: int64

<div class="alert alert-warning">
<font size="4"><b>⚠️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
<u>Контрольный вопрос:</u>

Не понял логику этого разделения. Можешь, пожалуйста, пояснить?

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Обратил внимание, что данные по второму региону состоят из двух легко разделяемых и одинаковых множеств. Разделил. Паразитные корреляции исчезли. RMSE стал ещё меньше. Во втором множестве оказались все 200 самых прибыльных скважин, что упрощает расчёты без потери соответствия требованиям задания. Если скважины второго подмножества локализованы (допускаю), то сразу понятно в какой части региона добыча более прибыльна и первое подмножество можно не рассматривать. Т.е. можно упростить, улучшить расчёты и, возможно, сразу определить более перспективный для добычи район региона №2.</font>
</div>

##### Подмножество 1

In [25]:
if RELEASE_GLOBAL:
    first_view(df2_1)

##### Подмножество 2

In [26]:
if RELEASE_GLOBAL:
    first_view(df2_2)

Наблюдение:

- датафрейм df2 состоит из двух равных подмножеств, на которые легко разделить по смежным категориям признака 'product'
- после разделения:
    - у признака 'f0' нормальное распределение
    - исчезли корреляции между 'f0', 'f1', 'product'
- у признака 'f1' нормальное распределение
- признак 'f2' имеет интервальный характер, а укрупнённо имеет категориальный и равномерное распределение
- признак 'product' имеет не интервальный, а категориальный характер и равномерное распределение
- корреляция 'f2' c 'product' около 1
- дубли в столбце 'id', возможно, возникли из-за коллизий фарш-функции

#### Третий регион

In [27]:
if RELEASE_GLOBAL:
    first_view(df3)

In [28]:
# Проверка в столбцах на уникальные значения, пропуски, дубликаты
for i in range(len(df3.columns)):
    column_value_test(df3.iloc[:, i], test='un')
    column_value_test(df3.iloc[:, i], test='na')
    column_value_test(df3.iloc[:, i], test='du')

Кол-во уникальных значений в столбце: "id": 99996/100000
Кол-во пропущенных значений в столбце: "id": 0/100000
Кол-во дубликатов в столбце:
id: 4/100000
Кол-во уникальных значений в столбце: "f0": 100000/100000
Кол-во пропущенных значений в столбце: "f0": 0/100000
Кол-во дубликатов в столбце:
f0: 0/100000
Кол-во уникальных значений в столбце: "f1": 100000/100000
Кол-во пропущенных значений в столбце: "f1": 0/100000
Кол-во дубликатов в столбце:
f1: 0/100000
Кол-во уникальных значений в столбце: "f2": 100000/100000
Кол-во пропущенных значений в столбце: "f2": 0/100000
Кол-во дубликатов в столбце:
f2: 0/100000
Кол-во уникальных значений в столбце: "product": 100000/100000
Кол-во пропущенных значений в столбце: "product": 0/100000
Кол-во дубликатов в столбце:
product: 0/100000


In [29]:
# дубли в столбце 'id'
df3[df3['id'].duplicated()].head()

,id,f0,f1,f2,product
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


In [30]:
df3.value_counts('product', normalize=False, sort=False)

product
0.000000      1
0.004606      1
0.009204      1
0.009761      1
0.014039      1
             ..
190.010029    1
190.010982    1
190.011722    1
190.013589    1
190.029838    1
Length: 100000, dtype: int64

Наблюдение:

- признаки 'f0', 'f1', 'f2' имеют нормальное распределение. Корреляции между собой нет
- признак 'product' имеет распределение смешанного вида (нормальное и равномерное)
- корреляция 'f2' и 'product' около 0,5
- дубли в столбце 'id', возможно, возникли в результате коллизий хеш-функции

In [31]:
# удаление признака 'id' из-за ненадобности
df1 = df1.drop('id', axis= 1)
df2 = df2.drop('id', axis= 1)
df3 = df3.drop('id', axis= 1)

df2_1 = df2_1.drop('id', axis= 1)
df2_2 = df2_2.drop('id', axis= 1)

#### Вывод

**Краткое описание:**

    - файлы данных:
        - 'https://code.s3.yandex.net/datasets/geo_data_0.csv'
        - 'https://code.s3.yandex.net/datasets/geo_data_1.csv'
        - 'https://code.s3.yandex.net/datasets/geo_data_2.csv'
    - в качестве разделителя в csv файле применён знак табуляции (',')
    - размерность: 5 столбцов на 100000 строк
    - типы данных: float64(4), object(1)

**Описание типов и распределеления данных:**

- geo_data_0.csv:
	- id      — [object]  — 
    - f0      — [float64] — вид сложения нескольких нормальных распределений
    - f1      — [float64] — вид сложения нескольких нормальных распределений
    - f2      — [float64] — нормальное распределение
    - product — [float64] — распределение смешанного вида (нормальное и равномерное)
    
    - корреляция между 'f0' и 'f1' около 0,5; eсть корреляции с 'product'
    - корреляция 'f2' c 'product' около 0,5
    
- geo_data_1.csv:
	- id      — [object]  — 
    - f0      — [float64] — имеет вид сложения двух нормальных распределений
    - f1      — [float64] — нормальное распределение
    - f2      — [float64] — укрупнённо принимает категориальный характер и равномерное распределение
    - product — [float64] — имеет не интервальный, а категориальный характер и равномерное распределение
    
    - датафрейм df2 легко можно разделить на два подмножества по смежным категориям признака 'product'
    - после разделения:
        - у признака 'f0' нормальное распределение
        - исчезают корреляции между 'f0', 'f1', 'product'
    - корреляция 'f2' c 'product' около 1
    
- geo_data_2.csv:
	- id      — [object]  — 
    - f0      — [float64] — нормальное распределение
    - f1      — [float64] — нормальное распределение
    - f2      — [float64] — нормальное распределение
    - product — [float64] — распределение смешанного вида (нормальное и равномерное)
    
    - корреляция 'f2' c 'product' около 0,5

**Пропуски, аномалии, дубликаты:**

количество пропусков:

    - все признаки   — 0/100000 
    
аномалии:
    
    - все признаки   — 0/100000 

количество уникальных значений в столбцах:

    - id      — 99990/100000 (df1), 99996/100000 (df2, df3)
    - f0      — 100000/100000
    - f1      — 100000/100000
    - f2      — 100000/100000
    - product — 100000/100000 (df1, df3), 12/100000 (df2)
    
количество дубликатов между столбцами:

    - не проверял

количество дубликатов строк с выборочными столбцами:

    - id, product: 0

**Прочее**:

    - дубли в столбце 'id', возможно, возникли в результае коллизий фарш-функции

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Здорово, что прежде чем обучать модели анализируешь данные, и делаешь это весьма добротно

## Обучение и проверка модели

In [32]:
def predict_oil(df, colname, title=''):     
    features_train, features_valid, target_train, target_valid = \
    split_stratified_into_train_val_test(df, stratify_colname=colname,
                                         frac_train=0.75, frac_valid=0.25, frac_test=0.0,
                                         random_state=12345)
    params = {}
    lr = LinearRegression()
    #gscv_lr = GridSearchCV(lr, param_grid = params, n_jobs = -1)
    model = lr.fit(features_train, target_train)
    predict = pd.Series(model.predict(features_valid))
    
    print()
    print(f'== {title} ==')
    print('Среднее значение разведанных запасов на скважину, тыс.бар.:  ', round(target_valid.mean(), 3))
    print('Среднее значение предсказанных запасов на скважину, тыс.бар: ', round(predict.mean(), 3))
    print('Среднеквадратичная ошибка RMSE: ', mean_squared_error(target_valid, predict) ** 0.5)
    print('\n')
    
    return predict, target_valid

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Молодец, что оформил в виде функции. Это в принципе правильный подход, повышающий простоту использования и поддержки кода, а когда вся работа по сути - это повторение однотипных действия над схожими датасетами, то это особенно актуально.

<div class="alert alert-info">
<font size="4">🍕<b> Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Настоятельно тебе рекомендую рандом стейт (и другие глобальные константы) в начале работы сохранять в отдельную переменную и оперировать дальше ей. Иногда бывает нужно провести эксперимент с другим рандомом и менять по коду во всех местах где он испоьзуется явно хуже, чем одну переменную в начале поменять.

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Сделал.</font>
</div>

In [33]:
# предикция запасов и разведанные запасы
predict1, target1 = predict_oil(df1, colname='product', title='Регион № 1')
predict2, target2 = predict_oil(df2, colname='product', title='Регион № 2')
predict3, target3 = predict_oil(df3, colname='product', title='Регион № 3')
predict2_1, target2_1 = predict_oil(df2_1, colname='product', title='Регион № 2.1')
predict2_2, target2_2 = predict_oil(df2_2, colname='product', title='Регион № 2.2')

dataframe:       (100000, 4)
features_train:  (75000, 3)
features_valid:  (25000, 3)
target_train:    (75000, 1)
target_valid:    (25000, 1)

== Регион № 1 ==
Среднее значение разведанных запасов на скважину, тыс.бар.:   92.079
Среднее значение предсказанных запасов на скважину, тыс.бар:  92.593
Среднеквадратичная ошибка RMSE:  37.5794217150813


dataframe:       (100000, 4)
features_train:  (75000, 3)
features_valid:  (25000, 3)
target_train:    (75000, 1)
target_valid:    (25000, 1)

== Регион № 2 ==
Среднее значение разведанных запасов на скважину, тыс.бар.:   68.723
Среднее значение предсказанных запасов на скважину, тыс.бар:  68.729
Среднеквадратичная ошибка RMSE:  0.893099286775617


dataframe:       (100000, 4)
features_train:  (75000, 3)
features_valid:  (25000, 3)
target_train:    (75000, 1)
target_valid:    (25000, 1)

== Регион № 3 ==
Среднее значение разведанных запасов на скважину, тыс.бар.:   94.884
Среднее значение предсказанных запасов на скважину, тыс.бар:  94.965
Сред

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Получены адекватные модели

Наблюдение:

- в регионе №3 наибольшие запасы, за ним регион №1 с небольшим отставанием
- для регионов №1 и №3 самые большие значения среднеквадратичной ошибки RMSE: 37.6 и 40
- меньше всего запасов в регионе №2, но среднеквадратичная ошибка менее 1
- а у подмножеств №2.1 и №2.2 среднеквадратичная ошибка менее 0,14 

## Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраним в отдельных переменных:

In [34]:
BUDGET_REGION = 10*(10**9)                     # 10 млрд.р. - бюджет разработки скважин в выбранном регионе
TESTED_WELLS = 500                             # количество разведочных скважин
SELECTED_WELLS = 200                           # количество скважин выбранных для разработки
BUDGET_PER_WELL = BUDGET_REGION/SELECTED_WELLS # бюджет на 1 скважину
REVENUE_BAR = 450                              # доход с каждого барреля
REVENUE_PRODUCT = REVENUE_BAR*(10**3)          # доход с каждой единицы продукта (1000 баррелей)
MAX_THRESHOLD_LOSS_PROBA = 0.025               # 2,5% - максимальный порог вероятности убытков
CONFIDENCE_LEVEL = 0.95                        # 0,95 - уровень доверия (1-уровень значимости)
QUANTILE_LOW = 0.025                           # 0,025 квантиль
QUANTILE_HIGH = 0.975                          # 0,975 квантиль

# требуемое количество нефти (тыс.бар.) с одной скважины для окупаемости 
MIN_VALUE_PRODUCT = BUDGET_PER_WELL / REVENUE_PRODUCT

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Здорово, что знаешь, что константы принято называть КАПСОМ

### Расчёт и сравнение достаточного объёма сырья для безубыточной разработки новой скважины

In [35]:
print('Затраты на скважину, млн.руб.:', round(BUDGET_PER_WELL/10**6, 6))
print('Требуемое кол-во нефти с одной скважины для окупаемости, тыс.бар.:', round(MIN_VALUE_PRODUCT, 3))

Затраты на скважину, млн.руб.: 50.0
Требуемое кол-во нефти с одной скважины для окупаемости, тыс.бар.: 111.111


<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Расчёт верный

In [36]:
# функция вычисления отклонения среднего значения запасов от значения окупаемости
def calc_deviation_stock(ds, title=''):
    deviation = round(ds.mean() - MIN_VALUE_PRODUCT, 3)
    print(f'{title} - отклонение ср. значения запасов скважины от окупаемости, тыс.бар.:', deviation)

# Отклонение среднего значения разведанных запасов по региону от значения запасов для окупаемости
calc_deviation_stock(target1, 'Регион № 1')
calc_deviation_stock(target2, 'Регион № 2')
calc_deviation_stock(target3, 'Регион № 3')
print()
calc_deviation_stock(target2_1, 'Регион № 2.1')
calc_deviation_stock(target2_2, 'Регион № 2.2')

Регион № 1 - отклонение ср. значения запасов скважины от окупаемости, тыс.бар.: -19.033
Регион № 2 - отклонение ср. значения запасов скважины от окупаемости, тыс.бар.: -42.388
Регион № 3 - отклонение ср. значения запасов скважины от окупаемости, тыс.бар.: -16.227

Регион № 2.1 - отклонение ср. значения запасов скважины от окупаемости, тыс.бар.: -43.765
Регион № 2.2 - отклонение ср. значения запасов скважины от окупаемости, тыс.бар.: -41.405


<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Молодец, что сравниваешь со средним таргетом, а не средним предсказанием. Тут у многих студентов логическая ошибка.

<div class="alert alert-warning">
<font size="4"><b>⚠️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
<u>Контрольный вопрос:</u>

При этом можно посмотреть, что средние таргеты очень близки средним предсказаниям во всех регионах, будь там высокое или низкое значение RMSE. То есть получается, что RMSE может быть высоким, а модель всё равно быть качественной? Или близость среднего предсказания среднему таргету не говорит о качестве модели?
    </font>
</div>

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Чем больше RMSE, тем больше разница между прогнозируемыми и наблюдаемыми значениями. Разница между не средними значениям выборок (хотя часто и по ним можно понять), а суммарная разница по всеми парам значений в выборках и далее по формуле RMSE.</font>
</div>

In [59]:
# если смотреть на разницу средних значений зависимость не всегда видна из-за взаимокомпенсирующих выбросов
print((predict1.mean()-target1.mean()), mean_squared_error(target1, predict1) ** 0.5)
print((predict2.mean()-target2.mean()), mean_squared_error(target2, predict2) ** 0.5)
print((predict3.mean()-target3.mean()), mean_squared_error(target3, predict3) ** 0.5)

0.5139710435510807 37.5794217150813
0.005410871086027669 0.893099286775617
0.0808131591505088 40.02970873393434


In [79]:
# если смотреть на разницу пар значений самих выборок всё сразу понятно
actual= [100, 53, 44, 47, 48, 48, 46, 43, 32, 27, 26, 24]
predict = [37, 40, 46, 44, 46, 50, 45, 44, 34, 30, 22, 100]

print(round(sum(actual)/len(predict), 3), round(sum(predict)/len(predict), 3))
print(mean_squared_error(actual, predict) ** 0.5)

44.833 44.833
28.818396901979124


### Выводы по этапу подготовки расчёта прибыли

- в каждом регионе средний запас сырья скважины меньше необходимого для безубыточной добычи на 19, 42.4, 16.2 тыс.бар.
- на основании предыдущего заключения можно отметить самую высокую убыточность разработки всех скважин в регионе № 2
- для получения прибыли будем исследовать в каждом регионе только 500 скважин, из которых выберем 200 самых прибыльных

## Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [47]:
# функция расчёта прибыли с заданным в SELECTED_WELLS количеством самых "богатых" скважин региона
def calc_profit(target, predictions):
    target = pd.Series(target).reset_index(drop=True)
    predict = pd.Series(predictions).reset_index(drop=True)
    predict_sorted = predict.sort_values(ascending=False)
    
    # выбранные скважины с максимальными предсказанными значениями запасов 
    selected = target[predict_sorted.index][:SELECTED_WELLS]
    
    # selected.sum() - целевое значение объёма сырья, соответствующее предсказаниям
    profit = selected.sum()*REVENUE_PRODUCT - BUDGET_REGION
    
    return profit

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Не знаю понимал ли ты что будет, если не сделать в первых строках функции сброс индексов, но в любом случае здорово, что смог избежать бага, на который попадаются почти все студенты! Его описание я в конце работы на всякий случай оставлю.

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Пришлось разобраться, т.к. без этого не работало --> "KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([ 9317,   219, 10015, 11584,  4296,\n            ...\n            14272, 23129, 21875, 14503, 20881],\n           dtype='int64', length=18853).".</font>
</div>

### Прибыль для полученного объёма сырья с лучших скважин

In [48]:
def former_profit(target, predictions, region='__'):  
    print(f'Регион № {region}, млрд.руб.: ',
          round(calc_profit(target, predictions)/10**9, 9))

print(f'== Прибыль с {SELECTED_WELLS} лучших скважин ==')

former_profit(target1, predict1, '1')
former_profit(target2, predict2, '2')
former_profit(target3, predict3, '3')
print()
former_profit(target2_1, predict2_1, '2_1')
former_profit(target2_2, predict2_2, '2_2')

== Прибыль с 200 лучших скважин ==
Регион № 1, млрд.руб.:  3.320826043
Регион № 2, млрд.руб.:  2.415086697
Регион № 3, млрд.руб.:  2.710349964

Регион № 2_1, млрд.руб.:  2.128967464
Регион № 2_2, млрд.руб.:  2.415086697


<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Мы сейчас видим, что во всех регионах прибыль. А вывод, что каждый регион имеет прибыль, на самом деле, для нас очень важен. И пока что не так важно где больше, это мы потом с помощью бутстрапа ещё поисследуем, а сейчас важно именно то, что гипотетически все в плюсе.<br>

Дело в том, что на текущем этапе, когда мы ещё не знаем что получится с помощью бутстрапа, мы уже можем оценить насколько хорошая картина нас может ждать. Ранее мы сравнили средние запасы регионов с точкой безубыточности, и увидели, что каждый регион в среднем убыточен. Если бы мы и здесь увидели убытки, то дальнейшая работа была бы бессмысленной. Зачем нам что-то считать, если у нас даже в лучшем случае убыток? А раз у нас возможна прибыль, то смысл есть, мы делаем качественную осмысленную работу, наша модель для бизнеса может быть полезна.<br>

В реальных проектах важно как можно раньше понять движемся ли мы в верном направлении или надо что-то менять. Потому что тратить время и деньги впустую - не лучшая затея.

Наблюдение:

- наибольшая прибыль в регионе №1, следом регион №3
- наименьшая прибыль в регионе №2
- прибыль подмножества №2.2 одинакова с регионом №2 в целом, значит все 200 самых "богатых" скважин региона в этом подмножестве 

## Расчёт прибыли и рисков 

Применим технику Bootstrap с 1000 выборок, чтобы найти:

- распределение прибыли
- среднюю прибыль
- 95% доверительный интервал
- 0,025 и 0,975 квантили распределения прибыли
- риск убытков

In [49]:
def bootstrap(target, predictions, title=''):  
    values = []
    state = RandomState(12345)
    for i in range(1000):
        target_subsample = target.sample(n=TESTED_WELLS, replace=True, random_state=state)
        predict_subsample = predictions[target_subsample.index]
        values.append(calc_profit(target_subsample, predict_subsample))
    
    values = pd.Series(values)
    lower = values.quantile(QUANTILE_LOW)
    higher = values.quantile(QUANTILE_HIGH)
    conf_int1, conf_int2 = st.t.interval(CONFIDENCE_LEVEL, df=len(values)-1,
                                         loc=values.mean(), scale=values.sem())
    
    print(f'== {title} =')
    print('Ожидаемая средняя прибыль, млн.руб.:', round(values.mean()/10**6, 6))
    print(f'95% доверительный интервал среднего, млн.руб.: {round(conf_int1/10**6, 6), round(conf_int2/10**6, 6)}')
    print('Квантиль 2,5% распределения прибыли, млн.руб.: ', round(lower/10**6, 6))
    print('Квантиль 97,5% распределения прибыли, млн.руб.:', round(higher/10**6, 6))
    print('Риск убытков, %:', round((values < 0).mean() * 100, 2))
    print()

In [50]:
bootstrap(target1.reset_index(drop=True), predict1, title='Регион № 1')
bootstrap(target2.reset_index(drop=True), predict2, title='Регион № 2')
bootstrap(target3.reset_index(drop=True), predict3, title='Регион № 3')

bootstrap(target2_1.reset_index(drop=True), pd.Series(predict2_1), title='Регион № 2.1')
bootstrap(target2_2.reset_index(drop=True), pd.Series(predict2_2), title='Регион № 2.2')

== Регион № 1 =
Ожидаемая средняя прибыль, млн.руб.: 396.164985
95% доверительный интервал среднего, млн.руб.: (379.620315, 412.709654)
Квантиль 2,5% распределения прибыли, млн.руб.:  -111.215546
Квантиль 97,5% распределения прибыли, млн.руб.: 909.766942
Риск убытков, %: 6.9

== Регион № 2 =
Ожидаемая средняя прибыль, млн.руб.: 456.045106
95% доверительный интервал среднего, млн.руб.: (443.147249, 468.942963)
Квантиль 2,5% распределения прибыли, млн.руб.:  33.820509
Квантиль 97,5% распределения прибыли, млн.руб.: 852.289454
Риск убытков, %: 1.5

== Регион № 3 =
Ожидаемая средняя прибыль, млн.руб.: 404.403867
95% доверительный интервал среднего, млн.руб.: (387.445797, 421.361936)
Квантиль 2,5% распределения прибыли, млн.руб.:  -163.350413
Квантиль 97,5% распределения прибыли, млн.руб.: 950.359575
Риск убытков, %: 7.6

== Регион № 2.1 =
Ожидаемая средняя прибыль, млн.руб.: 286.080019
95% доверительный интервал среднего, млн.руб.: (273.104061, 299.055977)
Квантиль 2,5% распределения прибы

<div class="alert alert-warning">
<font size="4"><b>⚠️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
<u>Контрольный вопрос:</u>

В чём разница между t-интервалом и интервала по квантилям? И какой из низ по идее запрашивает описание проекта?

Просто многие студенты вычисляют оба только потому что не понимают что от них хотят, вот мы и проверим понимание у тебя )
    </font>
</div>

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>В настоящем задании: "5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков..." В задании 3.6 спринта: "Постройте 95%-й доверительный интервал для среднего чека мармелада в интернет-магазине «Ползучая тянучка»...confidence_interval = st.t.interval(0.95, len(sample)-1, sample.mean(), sample.sem())". Думаю в текущем задании нужно сделать, как и в 3.6. Иначе я буду жаловаться:).
    95% доверительный интервал для среднего значения прибыли - интервал значений с 95% доверительной вероятностью перекрывающий часть диапазона распределения возможных средних значений прибыли, вокруг истинного среднего значения.
    Квантили 2,5% и 97,5% в настоящей работе задают доверительный интервал просто значений прибыли на интервале распределения всех значений прибыли. На всякий случай, чтобы зря не жаловаться:)
    </font>
</div>

Наблюдение:

- требуемый уровень риска убытков есть только у региона №2 и подмножества №2.2
- наибольшая ожидаемая средняя прибыль в регионе №2 (456 млн.руб.)
- ожидаемая средняя прибыль в регионе №3 (404 млн.руб.), регионе №1 (396 млн.руб.)
- средняя прибыль со скважин подмножества №2.2 (510 млн.руб.) больше чем в самом регионе №2 

### Выбор региона для разработки скважин

- регион №2 (geo_data_1.csv) единственный подходящий под требование "уровень риска менее 2,5%"
- если есть возможность локализовать скважины подмножества №2.2 из региона №2 это ещё больше улучшит показатели риска и средней прибыли

<div class="alert alert-info">
<font size="4">🍕<b> Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />
Подведём итоги. В целом всё очень даже хорошо, критических недочётов нет. Всё что жду - ответы на 3 контрольных вопроса, после которых принимаю работу.

## Общий вывод

Предобработка и анализ данных:

- регион №1 (geo_data_0.csv):
    
    - причудливые распределения, корреляции и диаграммы рассеяния
    - корреляция между 'f0' и 'f1' около 0,5; eсть корреляции с 'product'
    - корреляция 'f2' c 'product' около 0,5
    
- регион №2 (geo_data_1.csv):
    
    - датафрейм df2 легко можно разделить на два подмножества по смежным категориям признака 'product'
    - после разделения:
        - у признака 'f0' нормальное распределение
        - исчезают корреляции между 'f0', 'f1', 'product'
    - признак 'f2' укрупнённо принимает категориальный характер и равномерное распределение
    - признак 'product' имеет не интервальный, а категориальный характер и равномерное распределение
    - корреляция 'f2' c 'product' около 1
    
- регион №3 (geo_data_2.csv):

    - образцовое распределение данных признаков f0, f1, f2
    - корреляция 'f2' c 'product' около 0,5


Обучение и проверка модели:

- в регионе №3 наибольшие запасы, за ним регион №1 с небольшим отставанием
- для регионов №1 и №3 самые большие значения среднеквадратичной ошибки RMSE: 37.6 и 40
- меньше всего запасов в регионе №2, но среднеквадратичная ошибка менее 1
- а у подмножеств №2.1 и №2.2 среднеквадратичная ошибка менее 0,14 


Подготовка расчёта прибыли:

- в каждом регионе средний запас сырья скважины меньше необходимого для безубыточной добычи на 19, 42.4, 16.2 тыс.бар.
- на основании предыдущего заключения можно отметить самую высокую убыточность разработки всех скважин в регионе № 2
- для получения прибыли будем исследовать в каждом регионе только 500 скважин, из которых выберем 200 самых 


Расчёт прибыли для полученного объёма нефти с лучших скважин:

- наибольшая прибыль в регионе №1, следом регион №3
- наименьшая прибыль в регионе №2
- прибыль подмножества №2.2 одинакова с регионом №2 в целом, значит все 200 самых "богатых" скважин региона в этом подмножестве 


Расчёт прибыли и риска методом "Bootstrap":

- требуемый уровень риска убытков есть только у региона №2 и подмножества №2.2
- наибольшая ожидаемая средняя прибыль в регионе №2 (456 млн.руб.)
- ожидаемая средняя прибыль в регионе №3 (404 млн.руб.), регионе №1 (396 млн.руб.)
- средняя прибыль со скважин подмножества №2.2 (510 млн.руб.) больше чем в самом регионе №2 


ИТОГ:
- регион №2 (geo_data_1.csv) лучший из трёх и единственный подходящий по заданию для разработки нефтяных скважин
- если есть возможность локализовать скважины подмножества №2.2 из региона №2 это ещё больше улучшит показатели риска и средней прибыли

#### **ПРИЛОЖЕНИЕ**

Указанные выше выводы сделаны в результате следующих проделанных действий (кратко):

**Краткое описание:**

- файл данных:
    - 'https://code.s3.yandex.net/datasets/geo_data_0.csv'
    - 'https://code.s3.yandex.net/datasets/geo_data_1.csv'
    - 'https://code.s3.yandex.net/datasets/geo_data_2.csv'
- в качестве разделителя в csv файле применён знак табуляции (',')
- размерность: 14 столбцов на 10000 строк
- типы данных: float64(3), int64(8), object(3)

**Описание типов и распределеления данных:**

- df1 (geo_data_0.csv):
	- id      — [object]  — 
    - f0      — [float64] — вид сложения нескольких нормальных распределений
    - f1      — [float64] — вид сложения нескольких нормальных распределений
    - f2      — [float64] — нормальное распределение
    - product — [float64] — распределение смешанного вида (нормальное и равномерное)
    
    - корреляция между 'f0' и 'f1' около 0,5; eсть корреляции с 'product'
    - корреляция 'f2' c 'product' около 0,5
    
- df2 (geo_data_1.csv):
	- id      — [object]  — 
    - f0      — [float64] — имеет вид сложения двух нормальных распределений
    - f1      — [float64] — нормальное распределение
    - f2      — [float64] — укрупнённо принимает категориальный характер и равномерное распределение
    - product — [float64] — имеет не интервальный, а категориальный характер и равномерное распределение
    
    - датафрейм df2 легко можно разделить на два подмножества по смежным категориям признака 'product'
    - после разделения:
        - у признака 'f0' нормальное распределение
        - исчезают корреляции между 'f0', 'f1', 'product'
    - корреляция 'f2' c 'product' около 1
    
- df3 (geo_data_2.csv):
	- id      — [object]  — 
    - f0      — [float64] — нормальное распределение
    - f1      — [float64] — нормальное распределение
    - f2      — [float64] — нормальное распределение
    - product — [float64] — распределение смешанного вида (нормальное и равномерное)
    
    - корреляция 'f2' c 'product' около 0,5

**Изменение типов данных:**

- нет

**Пропуски, аномалии, дубликаты:**

пропуски:

- нет
    
аномалии:
    
- нет

количество уникальных значений в столбцах:

- id      — 99990/100000 (df1), 99996/100000 (df2, df3)
- f0      — 100000/100000
- f1      — 100000/100000
- f2      — 100000/100000
- product — 100000/100000 (df1, df3), 12/100000 (df2)

количество дубликатов между столбцами:

- не проверял

количество дубликатов строк с выборочными столбцами:

- id, product: 0


**Прочее**:

- дубли в столбце 'id', возможно, возникли в результае коллизий хеш-функции

**Добавлено типов данных:**
    
- нет
   
**Удалено типов данных:**
    
- столбец "id"

**Категоризация данных:**
    
- нет

**Проверка гипотез:**
    
- улучшение результатов при разделении массива данные из региона № 2 на два подмножества

**Итоговая структура данных:**
    
- без изменений
 
**Общий вывод**

**Трудоёмкость:**
   
- на выполнение работы затрачено полных 5 дней без перерыров
- дополнительно __ дня на исправление замечаний после ревью №1

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Дополнительный-инструментарий" data-toc-modified-id="Дополнительный-инструментарий-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Дополнительный инструментарий</a></span></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Первый-&quot;взгляд&quot;" data-toc-modified-id="Первый-&quot;взгляд&quot;-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Первый "взгляд"</a></span><ul class="toc-item"><li><span><a href="#Первый-регион" data-toc-modified-id="Первый-регион-0.3.1"><span class="toc-item-num">0.3.1&nbsp;&nbsp;</span>Первый регион</a></span></li><li><span><a href="#Второй-регион" data-toc-modified-id="Второй-регион-0.3.2"><span class="toc-item-num">0.3.2&nbsp;&nbsp;</span>Второй регион</a></span><ul class="toc-item"><li><span><a href="#Подмножество-1" data-toc-modified-id="Подмножество-1-0.3.2.1"><span class="toc-item-num">0.3.2.1&nbsp;&nbsp;</span>Подмножество 1</a></span></li><li><span><a href="#Подмножество-2" data-toc-modified-id="Подмножество-2-0.3.2.2"><span class="toc-item-num">0.3.2.2&nbsp;&nbsp;</span>Подмножество 2</a></span></li></ul></li><li><span><a href="#Третий-регион" data-toc-modified-id="Третий-регион-0.3.3"><span class="toc-item-num">0.3.3&nbsp;&nbsp;</span>Третий регион</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-0.3.4"><span class="toc-item-num">0.3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Расчёт-и-сравнение-достаточного-объёма-сырья-для-безубыточной-разработки-новой-скважины" data-toc-modified-id="Расчёт-и-сравнение-достаточного-объёма-сырья-для-безубыточной-разработки-новой-скважины-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Расчёт и сравнение достаточного объёма сырья для безубыточной разработки новой скважины</a></span></li><li><span><a href="#Выводы-по-этапу-подготовки-расчёта-прибыли" data-toc-modified-id="Выводы-по-этапу-подготовки-расчёта-прибыли-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Выводы по этапу подготовки расчёта прибыли</a></span></li></ul></li><li><span><a href="#Функция-для-расчёта-прибыли-по-выбранным-скважинам-и-предсказаниям-модели" data-toc-modified-id="Функция-для-расчёта-прибыли-по-выбранным-скважинам-и-предсказаниям-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели</a></span><ul class="toc-item"><li><span><a href="#Прибыль-для-полученного-объёма-сырья-с-лучших-скважин" data-toc-modified-id="Прибыль-для-полученного-объёма-сырья-с-лучших-скважин-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Прибыль для полученного объёма сырья с лучших скважин</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Выбор-региона-для-разработки-скважин" data-toc-modified-id="Выбор-региона-для-разработки-скважин-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Выбор региона для разработки скважин</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#ПРИЛОЖЕНИЕ" data-toc-modified-id="ПРИЛОЖЕНИЕ-5.0.1"><span class="toc-item-num">5.0.1&nbsp;&nbsp;</span><strong>ПРИЛОЖЕНИЕ</strong></a></span></li></ul></li></ul></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

<div class="alert alert-success">
<font size="4"><b>✔️ Комментарий ревьюера</b></font>
    <br /> 
    <font size="3", color = "black">
<br />    
Как и обещал, рассказ об одном очень популярном у студентов баге, которого ты избежал. Большинство студентов как и ты хранят и предсказания, и таргеты, как структуры pandas.Series. Сейчас на небольшом синтетеическом примере покажу что у них получается:
<br />

In [41]:
# КОД РЕВЬЮЕРА
# предположим, у нас есть небольшой сэмпл из 5 предсказаний и 5 соответствующих таргетов
_target = pd.Series([2, 2, 3, 3, 3], index=[2, 2, 3, 3, 3])
_preds = pd.Series([2, 2, 3, 3, 3], index=[2, 2, 3, 3, 3]).sort_values(ascending=False)


# берём таргеты по индексам предсказаний:

display(_target.loc[_preds.index])

3    3
3    3
3    3
3    3
3    3
3    3
3    3
3    3
3    3
2    2
2    2
2    2
2    2
dtype: int64

<div class="alert alert-success">
<font size="4"></font>
    <font size="3", color = "black">
Как так? Почему предсказаний 5, а таргетов аж 13? На самом деле, всё очень просто. На примере элементов с индексом и значением 2. Когда мы попросили у пандаса взять таргеты по индексам предсказаний, и у нас и там и там 2 одинаковых элемента, он отнюдь должен вернуть не 2. Он для каждого запрошенного индекса предсказаний выдаёт все таргеты с таким индексом. Таким образом, мы запросили 2 раза элемента с индексом 2 - оба раза пандас отдал нам 2 таких элемента из таргетов - всего 4 элемента. А если бы у нас было и там, и там по 3 одинаковых элемента, то в итоге было бы 9. Логика, надеюсь, ясна. И вот поэтому почти всегда у студентов получается, что когда берёшь таргеты по индексам 500 предсказаний, получают в итоге более чем 500 таргетов. Благодаря бутстрапу в выборках есть идентичные элементы, с одинаковыми индексами, и они каждый раз вот таким образом "множатся" при вычислении прибыли.

Сделать как у тебя, что одна из структур - не Series, а array (в array индексы повторяться не могут, поэтому проблема не возникает) - не единственное решение, их как минимум на 3 больше:

1. На самом деле, если внимательно посмотреть на нашу задачу, то становится понятно, что переменная target_subsample в функции bootstrap нам не нужна. Фактически она используется только для того, чтобы по её индексам взять сэмпл предсказаний. Но мы можем получить сэмпл предсказаний напрямую:

`pred_subsample = predictions.sample(n=500, replace=True, random_state=state)`

а в функцию расчёта прибыли передать pred_subsample и target (не сэмпл таргетов, а исходный, полный валидационный таргет, 25000 уникальных элементов с неповторяющимися индексами), и тогда получилось бы, что мы сэмплируем предсказания, сортируем их, выбираем 200 лучших, и только после этого по их индексам берём таргеты. Но поскольку таргеты бы были исходные, с уникальными индексами, то запросив один и тот же индекс 2 раза, мы бы в ответ получили 2 одинаковых элемента, а не 4, ошибки бы не было.

2. Можно было бы хранить предсказания и таргеты в одном датафрейме и оперировать бы с ним, а не отдельными его колонками. Тогда вообще не надо было бы делать операцию взятия по индексам: применил .sample() ко всему датафрейму, отсортировал по одной колонке, просуммировал первые 200 значений другой. Всё, элементы одной строки датафрейма между собой жёстко связаны, никакого расхождения не может быть, всё корректно.

3. Самое "лобовое" решение, которое большинство студентов и применяет: .reset_index(drop=True) и для таргетов, и для предсказаний в первых же строках функции расчёта прибыли. Логично: значения сохранились, индексы больше не повторяются, бага не будет:

In [42]:
# КОД РЕВЬЮЕРА
# предположим, у нас есть небольшой сэмпл из 5 предсказаний и 5 соответствующих таргетов
_target = pd.Series([2, 2, 3, 3, 3], index=[2, 2, 3, 3, 3]).reset_index(drop=True)
_preds = pd.Series([2, 2, 3, 3, 3], index=[2, 2, 3, 3, 3]).reset_index(drop=True).sort_values(ascending=False)


# берём таргеты по индексам предсказаний:

display(_target.loc[_preds.index])

2    3
3    3
4    3
0    2
1    2
dtype: int64